In [1]:
import pandas as pd
import win32com.client as win32
from datetime import datetime, timedelta
from datetime import date
import pytz

# Leer el archivo Excel
df = pd.read_excel('Eventos.xlsx')

# Inicializar la aplicación de Outlook
outlook = win32.Dispatch('outlook.application')


# Definir la zona horaria local
local_tz = pytz.timezone('America/Argentina/Buenos_Aires')

# Recorrer cada fila en el DataFrame
for index, row in df.iterrows():
    # Crear un nuevo evento en el calendario
    appointment = outlook.CreateItem(1) # 1 = olAppointmentItem
    appointment.Subject = row['Asunto']  # Ajusta esto según tus columnas
    
    # Combinar fecha y hora
    fecha_hora_str = f"{row['Fecha']} {row['Hora']}"
    # Convertir la cadena a datetime con el formato correcto
    fecha_hora = datetime.strptime(fecha_hora_str, "%d/%m/%Y %H:%M:%S")
    
    fecha_hora = fecha_hora - timedelta(hours=3)
    # Localizar la fecha y hora en la zona horaria local
    fecha_hora = local_tz.localize(fecha_hora)
    # Convertir a la zona horaria de UTC
    fecha_hora_utc = fecha_hora.astimezone(pytz.utc)
    appointment.Start = fecha_hora_utc
    
    appointment.Duration = 30           # Duración en minutos
    appointment.ReminderSet = True
    appointment.ReminderMinutesBeforeStart = 15  # Ajusta el tiempo de recordatorio
    appointment.Body = f"Recordatorio de vencimiento\n\nDetalles: {row['Detalles']}"

    # Añadir los destinatarios
    recipients = row['Destinatarios'].split(';')
    for recipient in recipients:
        appointment.Recipients.Add(recipient.strip())

    appointment.Save()

print("Eventos creados y correos enviados en Outlook.")


Eventos creados y correos enviados en Outlook.
